# Project configuration file

Since this file will also be uploaded, this avoids exposing sensitive data by configuration global paths. So define a project.ini file with a `images_folder` variable that will point towards an images folder called `fashion`.

In [1]:
import configparser
import pathlib

In [2]:
config_file = configparser.ConfigParser()
config_file.read('project.ini')

['project.ini']

In [3]:
IMAGES_FOLDER_PATH = pathlib.Path(config_file.get('default', 'images_folder'))


## Django setup

Initialize the Django backend

In [4]:
import os
import django
from django.conf import settings

In [5]:
BASE_DIR = pathlib.Path('.')

In [6]:
params = {
    'BASE_DIR': BASE_DIR,
    'INSTALLED_APPS': [
        'django.contrib.admin',
        'django.contrib.auth',
        'django.contrib.contenttypes',
        'django.contrib.sessions',
        'django.contrib.messages',
        'django.contrib.staticfiles',

        'django_extensions',
        'corsheaders',
        'rest_framework',
        'rest_framework.authtoken',
        
        'shop',
        'variants',
        'cart',
        'orders',
        'shipments',
        'reviews',
    ],
    'DATABASES': {
        'default': {
            'ENGINE': 'django.db.backends.postgresql',
            'NAME': 'mycommerce',
            'USER': 'test_user',
            'PASSWORD': 'touparet',
            'HOST': 'localhost',
            'PORT': '5432'
        }
    },
    'MEDIA_ROOT': BASE_DIR.joinpath('media'),
    'DEFAULT_AUTO_FIELD': 'django.db.models.BigAutoField',
    'PRODUCT_MODEL': 'shop.Product',
    'CUSTOMER_ORDERS_MODEL': 'orders.CustomerOrder',
    'STRIPE_TOKENS': [()]
}

try:
    settings.configure(**params)
except:
    pass

In [7]:
os.environ.setdefault('DJANGO_ALLOW_ASYNC_UNSAFE', 'True')

'True'

In [8]:
django.setup()

## File collection

Collect all the named images and videos in the images folder. The image's name will be used for the product's name. In that sense, similar images should have the same name in order for them to be considered the same product.

In [9]:
def collect_files():
    images = []
    videos = []

    if IMAGES_FOLDER_PATH.exists() and IMAGES_FOLDER_PATH.is_dir():
        files = IMAGES_FOLDER_PATH.glob('**/*')

        image_extensions = ['.jpg', '.jpeg']
        video_extensions = ['.mp4']
        
        images = list(filter(lambda x: x.suffix in image_extensions, files))
        videos = list(filter(lambda x: x.suffix in video_extensions, files))

    return images, videos

## Product creation and fixtures

Creates the related product for each image in the database backend. Creates a JSON fixture file for testing the proeject.

In [25]:
import json
import pathlib
import re
import random
import datetime
import pytz
import unidecode
import pandas
from urllib.parse import unquote
from shop.utils import create_slug
from shop.choices import ColorChoices, CategoryChoices
from django.utils.crypto import get_random_string

In [11]:
def detect_media_name(name):
    """Detects the true name of the media file"""
    if isinstance(name, pathlib.WindowsPath):
        name = unquote(str(name.stem))
        return name
    else:
        result = re.match(r'(\w.*)(?=\(\d+\))', name)
        if result is not None:
            return result.group(0).strip().lower()
    return get_random_string(10)

In [12]:
def create_product_fixtures(model_dotted_path=None, create_fixture=False):
    with open(BASE_DIR / 'fixtures.json', mode='w', encoding='utf-8') as f:
        products = []
        fixtures = []

        images, videos = collect_files()
        
        # Products created from image files
        # names = set(map(detect_media_name, images))

        with open(BASE_DIR / 'products_smky8.json', mode='r', encoding='utf-8') as b:
            data = json.load(b)
            names = list(map(lambda x: x['name'], data))

        timzone = pytz.timezone('America/Chicago')
        current_date = str(datetime.datetime.now(tz=timzone).date())
        
        for i, name in enumerate(names):
            if name is not None:
                color = str(random.choice(ColorChoices.choices)[0])
                
                product = {
                    'name': name.title(),
                    'color': color,
                    'unit_price': random.randrange(10, 800),
                    'active': False,
                    'display_new': random.choice([True, False]),
                    'slug': create_slug(f'{name}-{i}', color),
                    'category': str(random.choice(CategoryChoices.choices)[0]),
                    'created_on': current_date,
                    'modified_on': current_date
                }

                is_on_sale = random.choice([True, False])
                product['on_sale'] = is_on_sale
                if is_on_sale:
                    product['sale_value'] = random.randrange(10, 50)
                else:
                    product['sale_value'] = 0
                
                if create_fixture:
                    if model_dotted_path is None:
                        raise ValueError('Fixtures require a model path e.g. app.MyModel')
                    
                    fixtures.extend({
                        'pk': i + 1,
                        'model': model_dotted_path,
                        'fields': product
                    })

                    json.dump(fixtures, f)
                    print('Created', len(names), 'products')
                else:
                    products.append(product)
        return products

In [36]:
PRODUCTS = create_product_fixtures(model_dotted_path='shop.Product', create_fixture=False)

In [37]:
df = pandas.DataFrame(PRODUCTS)

In [44]:
def fix_tokens(value):
    tokens = {
        'LanièRes': 'Lanières',
        'PlisséE': 'Plissée',
        'ParéO': 'Paréo',
        'ChinéE': 'Chinée',
        'ÉLastique': 'Élastique',
        'RetrousséE': 'Retrousée'
    }
    
    clean_tokens = []
    for token in str(value).split(' '):
        if token in tokens:
            clean_tokens.append(tokens[token])
            continue
        clean_tokens.append(token)
    return ' '.join(clean_tokens)

df.name = df.name.apply(fix_tokens)

In [48]:
df = df.get(['name', 'color', 'unit_price', 'active', 'display_new', 'slug', 'category', 'on_sale', 'sale_value'])

In [49]:
df.head(n=3)

,name,color,unit_price,active,display_new,slug,category,on_sale,sale_value
0,Jupe Midi Popeline Taille Élastique,Yellow,337,False,True,jupe-midi-popeline-taille-elastique-0-yellow,Activewear,False,0
1,Minijupe Cargo Serge Ceinture,Camel,710,False,True,minijupe-cargo-serge-ceinture-1-camel,Bags,True,42
2,Jupe-Culotte Paréo Plissée,Red,57,False,False,jupe-culotte-pareo-plissee-2-red,Pants,False,0


In [50]:
df.to_csv('products.csv', index=False)

## Database object creation

Save all of the items in the database

In [18]:
from shop.models import Image, Video, Product
from shop.utils import process_file_name
from django.contrib.auth import get_user_model
from django.core.files import File


In [19]:
def create_media_objects(commit=False, images_alone=False):
    images, videos = collect_files()
    
    def image_iterator():
        for i, image in enumerate(images):
            f = open(image, mode='rb')
            clean_name = detect_media_name(image.name)
            yield Image(name=f"{clean_name}{i}", original=File(f, name=image.name))
            
    def video_iterator():
        for i, video in enumerate(videos):
            f = open(video, mode='rb')
            clean_name = detect_media_name(video.name)
            yield Video(name=f"{clean_name}{i}", content=File(f, name=video.name))
    
    if commit:
        Image.objects.bulk_create(image_iterator())
        
        if not images_alone:
            Video.objects.bulk_create(video_iterator())
    else:
        return image_iterator(), video_iterator()

In [20]:
def create_products(commit=False):
    if commit:
        for data in PRODUCTS:
            Product.objects.get_or_create(
                name=data.pop('name'),
                color=data.pop('color'),
                defaults=data
            )

    # def iterator():
    #     for data in PRODUCTS:
    #         yield Product(**data)
    # if commit:
    #     Product.objects.bulk_create(iterator())

In [21]:
create_media_objects(commit=True, images_alone=True)

In [22]:
create_products(commit=True)

IntegrityError: la valeur d'une clé dupliquée rompt la contrainte unique « shop_product_slug_key »
DETAIL:  La clé « (slug)=(jupe-midi-popeline-taille-elastique-pink) » existe déjà.

In [ ]:
USER_MODEL = get_user_model()

def create_users():
    users = [
        ['lucile@gmail.com', 'Lucile', 'Lopez', 'touparette'],
        ['pauline@gmail.com', 'Pauline', 'Matthieu', 'touparette']
    ]
    for user in users:
        try:
            user_object = USER_MODEL.objects.create_user(email=user[0], password=user[-1], first_name=user[1], last_name=user[2])
        except:
            print('User exists already')